In [ ]:
# CFA to IFA
import torch
import torch.nn as nn
from collections import OrderedDict

CFA_model_path = '' # CFA model path
a = torch.load(CFA_model_path)

tmp = []
for k in a['model']:
    # print(k)
    if k.split('.')[0] == 'uise_neck' and k.split('.')[1] == 'deconv' and k.split('.')[2][:4] == 'conv':
        tmp.append(k)
        print(k)

weights = torch.cat( (a['model'][tmp[0]], a['model'][tmp[1]], a['model'][tmp[2]], a['model'][tmp[3]]), dim = 1 )
bias = a['model'].pop('uise_neck.deconv.bias')

for k in tmp:
    print(a['model'][k].shape)
    a['model'].pop(k)

a['model']['uise_neck.deconv.fuse_conv.bias'] = bias.reshape(-1)
a['model']['uise_neck.deconv.fuse_conv.weight'] = weights

IFA_model_path = '' # IFA model path
torch.save(a, IFA_model_path)

In [ ]:
# IFA to CFA
import torch
import torch.nn as nn
from collections import OrderedDict

IFA_model_path = ''
a = torch.load(IFA_model_path)

weight = a['model'].pop('uise_neck.deconv.fuse_conv.weight')
bias = a['model'].pop('uise_neck.deconv.fuse_conv.bias')

out_channels=[1024, 512, 256, 128]
a['model']['uise_neck.deconv.conv_a5.weight'] = weight[:, : out_channels[0] ]
a['model']['uise_neck.deconv.conv_a4.weight'] = weight[:, sum(out_channels[:1]) : sum(out_channels[:2])]
a['model']['uise_neck.deconv.conv_a3.weight'] = weight[:, sum(out_channels[:2]) : sum(out_channels[:3])]
a['model']['uise_neck.deconv.conv_a2.weight'] = weight[:, sum(out_channels[:3]) : ]
a['model']['uise_neck.deconv.bias'] = bias.reshape(1, -1, 1, 1)

CFA_model_path = '' # CFA model path
torch.save(a, CFA_model_path)